<a href="https://colab.research.google.com/github/wallik2/Optimize_using_genetic_algorithm/blob/main/%5BGitHub%5D_Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

dataset : https://data.world/adamhelsinger/food-nutrition-information/workspace/file?filename=NutritionalFacts_Fruit_Vegetables_Seafood.csv

more learning resource : https://www.obitko.com/tutorials/genetic-algorithms/crossover-mutation.php

<fieldset>

**Story** :

Imagine you have 40 limited food items and you want to be healthy by maximize the fiber per day

However, you only can hold for 8 foods to make a meal for some reason, also you don't need to consume sugar exceed than 25g 


<fieldset>


Problem :

Data: We import from *data.world* which contain the nutrient for each food

**constraint** :<br> 1) the sum of the sugar from all of 8 foods is not higher than 25g
           <br> 2) Cannot contain the duplicated food
          
**onjective function** : Maximizing the fiber (g)
<br><br>
**Fitness function** : the threshold update everytime with min(fiber) + 0.8(max(fiber)-min(fiber))

Since we don't know the optimal solution, so I decided to stop iterate more gene pool when the max fiber stay for 100 iterations



---



# 1. Preprocessing data

In [1]:
import random, copy, math, numpy as np, pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/wallik2/Optimize_using_genetic_algorithm/main/NutritionalFacts_Fruit_Vegetables_Seafood.csv'

raw = pd.read_csv(url, encoding='latin-1')

print("setup completed")

setup completed


In [4]:
raw.head(4)

,Food and Serving,Calories,CaloriesÃfrom Fat,Total Fat,Total Fat.1,Sodium,Sodium.1,Potassium,Potassium.1,Total Carbo-hydrate,Total Carbo-hydrate.1,Dietary Fiber,Dietary Fiber.1,Sugars,Protein,Vitamin A,Vitamin C,Calcium,ÃÃIronÃÃ,Saturated Fat,Saturated Fat.1,Chole-sterol,Chole-sterol.1,Food Type
0,NaN,NaN,NaN,(g),(%DV),(g),(%DV),(g),(%DV),(g),(%DV),(g),(%DV),(g),(g),(%DV),(%DV),(%DV),(%DV),(%DV),(mg)Ã,(%DV),(mg)Ã,NaN
1,"Asparagus, 5 spears (93 g/3.3 oz)",20.0,0.0,0,0,0,0,230,7,4,1,2,8,2,2,10,15,2,2,NaN,NaN,NaN,NaN,"Vegetables, Serving Size (gram weight/Ãounce ..."
2,"Bell Pepper, 1 medium (148 g/5.3 oz)",25.0,0.0,0,0,40,2,220,6,6,2,2,8,4,1,4,190,2,4,NaN,NaN,NaN,NaN,"Vegetables, Serving Size (gram weight/Ãounce ..."
3,"Broccoli, 1 medium stalk (148 g/5.3 oz)",45.0,0.0,0.5,1,80,3,460,13,8,3,3,12,2,4,6,220,6,6,NaN,NaN,NaN,NaN,"Vegetables, Serving Size (gram weight/Ãounce ..."


In [ ]:
raw.drop(0,axis=0,inplace=True)

In [ ]:
# the purpose is to show the food sequence.

raw_food = raw.loc[:,['Food and Serving','Dietary Fiber','Sugars']].dropna(axis=0);#raw_food

In [ ]:
def pre_data(df,obj,constraint,Drop=True):
  df = df.loc[:,[obj,constraint]]
  if Drop==True:
    df.dropna(inplace=True)
    df.reset_index(drop=True,inplace=True)
  return df

In [ ]:
df = pre_data(raw,'Dietary Fiber','Sugars');#df



---



# 2. Setup Genetic Algorithm (GA)

select the best 8, with highest fiber and within limited sugar

In [ ]:

"""setup parameter"""
max_iteration = 100

# we limit only 8 foods
length = 8

#we limit the sugar not more than 25
sugars_limit = 25

#gene sample space 
charset = list(df.index)

Note : This is **Permutation Encoding**

In [ ]:
# check how correct the match of the random word sequence and the true word sequence 
def random_string():
  return random.sample(charset, length)

def evaluation(input_string):
    
    result = df.iloc[input_string].astype(float).sum(axis=0)
    fiber = result['Dietary Fiber']
    sugar = result['Sugars']
    #print(fiber,sugar)
    if sugar > sugars_limit:
      #ignore the seq of food exceeding sugar limit
      return 0 

    else:
      return fiber    


def crossover(str1, str2):
    r = random.randint(0, length-1)
    #print(str1,str2)
    #Prevent duplicate gene
    dup = list(set(str1) & set(str2))
    
    new_string = []
    for i in range(length):

        if (str1[i] in dup) or (str2[i] in dup):
          new_string.append(str1[i])
        elif i < r:
            new_string.append(str1[i])
        else:
            new_string.append(str2[i])

           
    return new_string


def mutation(input_string):

    #Filter the possible mutation (avoid the duplicate index in list)
    unwanted = input_string
    item_list = [e for e in charset if e not in unwanted]

    new_string = []
    r = random.randint(0, length-1)
    c = random.choice(item_list)
    
    #length=8
    for i in range(length):

      #Very low chance to get mutated (1/length)
        if i != r:
            #new_string += input_string[i]
            new_string.append(input_string[i])

        # append new random    
        else:
            #new_string+= c
            new_string.append(c)

    return new_string 

def fill_pool(pool, n):
    for i in range(n):
        s = random_string()
        pool.append(s)
        #print(pool)
        if evaluation(s) > 100:
            print('found word ', s)
            break
        
    return pool


# Mutate N gene in the pool

def mutate_pool(pool,n):
    for i  in range(n):
        string = random.choice(pool)
        m_string = mutation(string)
        if evaluation(m_string) > evaluation(string):
            pool.append(m_string)
    return pool


def crossover_pool(pool,n):
    for i in range(n):
        str1 = random.choice(pool)
        str2 = random.choice(pool)
        #print(str1,str2)

        str3 = crossover(str1, str2)
        #print(str3)

        if evaluation(str3) > max(evaluation(str1),evaluation(str2)):
            pool.append(str3)
    return(pool)


def prune_pool(pool,r):
    # remain first n best items (prune = ตัด)
    
    
    min_eval = math.inf
    max_eval = 0

    # find min and max
    for i in range(int(0.1*len(pool))):
        eval_string = evaluation(pool[i])
    #update parameter (max,min)
        if eval_string <= min_eval:
            min_eval = eval_string
        if eval_string > max_eval :
            max_eval = eval_string
            #update the best seq
            globals()['max_string'] = pool[i]

            
    threshold = min_eval + r*(max_eval - min_eval)
    #declare the threshold
    print(max_eval, min_eval, threshold)
    
    globals()['max_fiber'] = max_eval

    # now select the fit one
    new_pool = []
    
    for string in pool:
        eval_string = evaluation(string)
        if eval_string >= threshold:
            new_pool.append(string)


    pool = new_pool
    return pool
        
            



---



In [ ]:
"""

def GA(iteration,sugar_limits_,food_menu_):
  #print(display(food_menu_))
  #Setup
  global sugar_limits, length, df, charset

  (sugar_limits , df) = (sugar_limits_ , food_menu_)
  length = len(food_menu_)

  charset = list(df.index);
  print(sugar_limits,length,charset)

  #globals()['max_fiber'] = max_eval



  #define pool here
  globals()['max_fiber'] = 0
  pool = []
  streak=0
  print('START')
  while any_solution(pool,iteration,streak) < iteration:

      print('checked')
      pool = fill_pool(pool, 100)
      print('After filling ', len(pool))
        
      pool = mutate_pool(pool,500)
      print('After mutation ',len(pool))
        
      pool = crossover_pool(pool,2000)
      print('After crossing over ',len(pool))
        
      pool = prune_pool(pool, 0.8)
      print('After pruning ',len(pool))
      
      streak = any_solution(pool,iteration,streak)




#GA(iteration = 100,25,df)
"""

"\n\ndef GA(iteration,sugar_limits_,food_menu_):\n  #print(display(food_menu_))\n  #Setup\n  global sugar_limits, length, df, charset\n\n  (sugar_limits , df) = (sugar_limits_ , food_menu_)\n  length = len(food_menu_)\n\n  charset = list(df.index);\n  print(sugar_limits,length,charset)\n\n  #globals()['max_fiber'] = max_eval\n\n\n\n  #define pool here\n  globals()['max_fiber'] = 0\n  pool = []\n  streak=0\n  print('START')\n  while any_solution(pool,iteration,streak) < iteration:\n\n      print('checked')\n      pool = fill_pool(pool, 100)\n      print('After filling ', len(pool))\n        \n      pool = mutate_pool(pool,500)\n      print('After mutation ',len(pool))\n        \n      pool = crossover_pool(pool,2000)\n      print('After crossing over ',len(pool))\n        \n      pool = prune_pool(pool, 0.8)\n      print('After pruning ',len(pool))\n      \n      streak = any_solution(pool,iteration,streak)\n\n\n\n\n#GA(iteration = 100,25,df)\n"



---



<fieldset>

**FLEETING**

Future development : 

1.Should do OOP for easier to manipulate 

2. Make less verbal GA(verbose=False) https://stackoverflow.com/questions/8391411/how-to-block-calls-to-print   + with progress bar (tqdm)

3. minimize sugar once max_fiber is reached

4. give the name of foods (dictionary)

In [ ]:
"""Launch"""

solution = GA(iteration=max_iteration)

START
max fiber 0
checked
After filling  100
After mutation  102
After crossing over  111
0 0 0.0
After pruning  111
max fiber 0
found  [22, 27, 11, 14, 7, 8, 30, 23]
found  [7, 35, 4, 8, 27, 10, 15, 33]
found  [0, 25, 19, 17, 15, 23, 22, 24]
found  [11, 13, 18, 19, 29, 35, 1, 21]
found  [3, 29, 22, 37, 11, 28, 9, 39]
found  [9, 10, 27, 3, 0, 25, 31, 30]
found  [7, 30, 8, 35, 29, 18, 22, 24]
found  [37, 24, 26, 32, 29, 22, 21, 30]
found  [29, 12, 26, 5, 9, 24, 38, 32]
found  [25, 13, 39, 16, 3, 6, 15, 22]
found  [38, 11, 26, 9, 15, 2, 21, 33]
found  [13, 10, 35, 14, 6, 21, 38, 25]
found  [1, 30, 24, 38, 10, 6, 8, 23]
found  [17, 14, 27, 19, 6, 35, 28, 12]
found  [7, 38, 2, 12, 34, 22, 14, 27]
found  [25, 28, 37, 20, 29, 31, 9, 13]
found  [12, 18, 32, 20, 24, 36, 25, 2]
found  [39, 33, 16, 9, 19, 13, 11, 6]
found  [30, 4, 25, 8, 16, 3, 23, 6]
found  [32, 1, 30, 38, 18, 14, 16, 20]
found  [37, 14, 9, 3, 17, 12, 39, 7]
found  [8, 13, 15, 19, 21, 20, 11, 18]
found  [30, 1, 35, 38, 0, 36, 1

,Dietary Fiber,Sugars
4,2,2
18,4,7
16,2,2
2,3,2
7,3,2
8,2,3
5,2,2
28,2,2


With the fiber : 20.0 g , sugar : 22.0  g



which contain
Cauliflower, 1/6 medium head (99 g/3.5 oz)
Sweet Potato, 1 medium, 5" long,Ê2" diameter (130 g/4.6 oz)
SummerÊSquash, 1/2 medium (98 g/3.5 oz)
Broccoli, 1 medium stalk (148 g/5.3 oz)
Green (Snap) Beans, 3/4 cup cut (83 g/3.0 oz)
GreenÊCabbage, 1/12 medium head (84 g/3.0 oz)
Celery, 2 medium stalks (110 g/3.9 oz)
Lemon, 1 medium (58 g/2.1 oz)




---



sandbox

In [ ]:
display(solution[0])

print(f'With the fiber : {solution[1]} g , sugar : {solution[2]}  g')


,Dietary Fiber,Sugars
4,2,2
18,4,7
16,2,2
2,3,2
7,3,2
8,2,3
5,2,2
28,2,2


With the fiber : 20.0 g , sugar : 22.0  g


Note that it might not be a global solution, so the higher iteration you set, the high chance you obtain global solution

In [ ]:
df.iloc[max_string]

print(f'With the fiber : {solution[1]} g , sugar : {solution[2]}  g')


With the fiber : 20.0 g , sugar : 22.0  g
